In [7]:
import os
import sys
path = !echo ${VIRTUAL_ENV}
path = os.path.join(path[0], '..')
sys.path.append(path)
from helper.utils import load_from_pickle, load_from_shelve
import numpy as np
import pandas as pd
from plotly import tools
import plotly as py
import plotly.graph_objs as go
#import impywidgets as widget
py.offline.init_notebook_mode(connected=True)

In [8]:
base_data_path = "/home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/model_performances/subset_5000/"

In [19]:
debug_files = {}
for i, structure in enumerate(os.walk(base_data_path)):
    root, dirs, files = structure
    for file in files:
        if file.endswith(".pkl"):            
            debug_files[i] = os.path.join(root, file)
["{} --> {}".format(key, value) for key, value in debug_files.items()]

['1 --> /home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/model_performances/subset_5000/model_2_layers_6_epoch_100_mar_0.6_sf_0.1_0.1_0.1_0.1_0.1_0.1_wd_0.001_0.001_0.001_0.001_0.001_0.001_lr_0.001_dim_2048_2048_2048_2048_2048_2048_keep_1_1_1_1_1_1_seed_66_66_66_66_66_66_loss_v3/debug_dict.pkl',
 '2 --> /home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/model_performances/subset_5000/ELMO_model_2_layers_6_epoch_100_mar_0.6_sf_0.1_0.1_0.1_0.1_0.1_0.1_wd_0.001_0.001_0.001_0.001_0.001_0.001_lr_0.001_dim_2048_2048_2048_2048_2048_2048_keep_1_1_1_1_1_1_seed_66_66_66_66_66_66_loss_v3/debug_dict.pkl']

In [20]:
model_indx =1
my_dict = load_from_pickle(os.path.join(base_data_path, 
                                        debug_files[model_indx]))
num_of_epochs = my_dict['epochs']
number_of_sample_question_size = 15
# temp_ids = np.arange(my_dict['actual_labels'].shape[0])
# np.random.shuffle(temp_ids)
# question_ids = temp_ids[:number_of_sample_question_size]
debug_files[model_indx]

'/home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/model_performances/subset_5000/model_2_layers_6_epoch_100_mar_0.6_sf_0.1_0.1_0.1_0.1_0.1_0.1_wd_0.001_0.001_0.001_0.001_0.001_0.001_lr_0.001_dim_2048_2048_2048_2048_2048_2048_keep_1_1_1_1_1_1_seed_66_66_66_66_66_66_loss_v3/debug_dict.pkl'

In [21]:
"""
How the recalls at top n are changing for each epoch 
"""

data_recalls = []
sub_list = []
sub_list.append(0)
sub_list.extend([x for x in my_dict['normalized_recalls_before_model']])
data_recalls.append(sub_list)
for i in range(1,num_of_epochs+1):
    sub_list = []
    sub_list.append(i)
    sub_list.extend([x for x in my_dict[str(i)]['normalized_recalls_after_model']])
    data_recalls.append(sub_list)
df_data_recalls = pd.DataFrame(data_recalls, columns=['epoch', 'top_1', 'top_2', 'top_5'
                                                   ,'top_10', 'top_20', 'top_50'])
traces = []
for c in df_data_recalls.columns[1:]:
    trace = go.Scatter(
    x = df_data_recalls.index,
    y = df_data_recalls[c],
    mode = 'lines+markers',
    name = 'Top {}'.format(c))
    traces.append(trace)

layout = go.Layout(
    title='How the recalls at top n are changing for each epoch',
    xaxis=dict(
        title='Epochs'
    ),
    yaxis=dict(
        title='Recall Values'
    )
)
fig =  go.Figure(data=traces, layout=layout)
py.offline.iplot(fig)

In [22]:
df_data_recalls[['top_1', 'top_2', 'top_5','top_10', 'top_20', 'top_50']].mean(axis=1).sort_values(ascending =[False])

90     0.093933
89     0.093900
87     0.093867
82     0.093767
96     0.093767
91     0.093667
88     0.093667
85     0.093633
98     0.093633
92     0.093600
86     0.093533
93     0.093467
99     0.093467
83     0.093400
94     0.093400
97     0.093300
95     0.093200
100    0.093067
80     0.093033
79     0.092900
84     0.092867
78     0.092833
81     0.092667
77     0.092367
76     0.092267
73     0.092233
70     0.092200
72     0.092000
75     0.091967
66     0.091700
         ...   
30     0.086800
28     0.086400
26     0.085867
27     0.085800
23     0.085467
25     0.085400
24     0.085333
22     0.084667
21     0.084367
20     0.083633
19     0.083267
17     0.082467
18     0.082367
16     0.082233
15     0.080833
14     0.080333
13     0.079333
12     0.079167
11     0.078633
10     0.078033
9      0.077633
8      0.077033
7      0.074533
6      0.073100
5      0.069500
4      0.065167
3      0.059467
2      0.050900
1      0.034700
0      0.004333
Length: 101, dtype: floa

In [23]:
df_data_recalls.sort_values(by=['top_1', 'top_2', 'top_5', 'top_10', 'top_20', 'top_50'], ascending =[False, False, False, False, False, False])

,epoch,top_1,top_2,top_5,top_10,top_20,top_50
59,59,0.0242,0.0388,0.0628,0.0940,0.1322,0.1962
87,87,0.0242,0.0384,0.0664,0.0976,0.1360,0.2006
47,47,0.0242,0.0382,0.0632,0.0914,0.1312,0.1924
85,85,0.0242,0.0374,0.0658,0.0960,0.1370,0.2014
63,63,0.0240,0.0388,0.0638,0.0948,0.1332,0.1948
42,42,0.0240,0.0372,0.0618,0.0908,0.1280,0.1908
89,89,0.0238,0.0390,0.0666,0.0982,0.1352,0.2006
83,83,0.0238,0.0388,0.0660,0.0970,0.1344,0.2004
48,48,0.0238,0.0388,0.0624,0.0918,0.1312,0.1932
91,91,0.0238,0.0380,0.0666,0.0974,0.1346,0.2016


In [43]:
"""
which of questions are getting worse (moving farther from its ground-truth) through subsequent epochs.
"""

k = 1
good_questions_at_top_1_idx = np.where(my_dict['top_k'][str(k)]['are_founds_before'] == 1)
np.random.shuffle(good_questions_at_top_1_idx[0])
# IN ORDER TO SELECT RANDOM QUESTIONS
#question_ids = good_questions_at_top_1_idx[0][:number_of_sample_question_size]

# IN ORDER TO ENTER QUESTIONS
question_ids = [1550, 4257, 459]

good_questions_at_top_1_are_founds = my_dict['top_k'][str(k)]['are_founds_before'][question_ids]
good_questions_at_top_1_labels = my_dict['top_k'][str(k)]['closest_labels_before'][question_ids]
good_questions_at_top_1_distances = my_dict['top_k'][str(k)]['distances_before'][question_ids]

still_good_question = pd.DataFrame()
still_good_question[0]= np.reshape(good_questions_at_top_1_are_founds, [-1,])
for i in range(1,num_of_epochs+1):
    still_good_question[i] = np.reshape(my_dict[str(i)]['top_k'][str(k)]['are_founds_after'][question_ids], [-1,])
still_good_question = still_good_question.T
still_good_question.columns = [question_ids[col_name] for col_name in still_good_question.columns]
data = []
problematic_questions = []
for q in question_ids:
    _problematic_question = np.where(still_good_question[q] == 0)[0]
    _problematic_question = still_good_question[q][_problematic_question]
    if _problematic_question.shape[0] != 0:
        problematic_questions.append(_problematic_question.name)
    trace = go.Scatter(
    x = still_good_question.index,
    y = still_good_question[q],
    mode = 'lines+markers',
    name = 'Question {}'.format(q))
    data.append(trace)

layout = go.Layout(
    title='Questions" Paragraphs are still in their top {} closest neighbours'.format(k),
    xaxis=dict(
        title='Epochs'
    ),
    yaxis=dict(
        title='Yes or No'
    )
)
fig =  go.Figure(data=data, layout=layout)
py.offline.iplot(fig)
print('Problematic Questions')
[_p for _p in problematic_questions]

Problematic Questions


[4257]

In [44]:
"What happened to the following question?"
sample_question_that_are_getting_worse_idx = problematic_questions
sample_question_that_are_getting_worse_sub_idx = [np.where(question_ids==q)[0][0] for q in sample_question_that_are_getting_worse_idx]
#label_of_sample_question = good_questions_at_top_1_labels[sample_question_that_are_getting_worse_sub_idx]
"Is that question getting farther from its ground-truth paragraph?"
getting_worse_distances = pd.DataFrame()
getting_worse_labels = pd.DataFrame()
getting_worse_distances[0] = np.reshape(good_questions_at_top_1_distances[sample_question_that_are_getting_worse_sub_idx], [-1,])
getting_worse_labels[0] = np.reshape(good_questions_at_top_1_labels[sample_question_that_are_getting_worse_sub_idx], [-1,])
for i in range(1,num_of_epochs+1):
    getting_worse_distances[i] = np.reshape(my_dict[str(i)]['top_k'][str(1)]['distances_after'][sample_question_that_are_getting_worse_idx], [-1,])
    getting_worse_labels[i] = np.reshape(my_dict[str(i)]['top_k'][str(1)]['closest_labels_after'][sample_question_that_are_getting_worse_idx], [-1,])

getting_worse_distances = getting_worse_distances.T
getting_worse_labels = getting_worse_labels.T
getting_worse_distances.columns = [sample_question_that_are_getting_worse_idx[i] for i, col_name in enumerate(getting_worse_distances.columns)]
getting_worse_labels.columns = [sample_question_that_are_getting_worse_idx[i] for i, col_name in enumerate(getting_worse_labels.columns)]

data = []
for q in sample_question_that_are_getting_worse_idx:
    trace = go.Scatter(
    x = getting_worse_distances.index,
    y = getting_worse_distances[q],
    text = getting_worse_labels[q],
    hoverinfo = 'y' 'text',
    mode = 'lines+markers',
    name = 'Question {}'.format(q))
    data.append(trace)

layout = go.Layout(
    title='Are questions getting farther from their ground-truth paragraph in each epoch iteration? <br> ' +
    'to grab the closest paragraph, please hover the question',
    xaxis=dict(
        title='Epochs'
    ),
    yaxis=dict(
        title='Distance to Closest Paragraph'
    )
)
fig =  go.Figure(data=data, layout=layout)
py.offline.iplot(fig)
closest_paragraphs_of_problematic_questions = dict()
print('Closest Paragraphs of these Problematic Questions')
for _q in problematic_questions:
    closest_paragraphs_of_problematic_questions[_q] = set(getting_worse_labels[_q]) 
closest_paragraphs_of_problematic_questions

Closest Paragraphs of these Problematic Questions


{4257: {16065, 18823}}

In [45]:
total_ks = len(my_dict['top_k'])
questions = []
for question, paragraphs in closest_paragraphs_of_problematic_questions.items():
    print('*' * 10)
    print('Question {}'.format(question))    
    # iterate each k for that question so that we can specify 
    # what are the closest paragraph to that question for each k, what is their distance
    # and also what is the order of ground truth paragraph to that question  within the list of results in k and 
    # its distance
    
    # FOR EPOCH 0 : BEFORE APPENDIX
    _epoch = 0
    for _k in range(total_ks):
        paragraph_at_k = my_dict['top_k'][str(total_ks)]['closest_labels_before'][question][_k]
        q_to_p_distance_at_k = my_dict['top_k'][str(total_ks)]['distances_before'][question][_k]
        is_ground_truth_in_k_range = my_dict['top_k'][str(_k+1)]['are_founds_before'][question][0]
        questions.append((question,
                         _epoch,
                         _k+1,
                         paragraph_at_k, 
                         q_to_p_distance_at_k,
                         is_ground_truth_in_k_range
        ))
    # FOR EPOCH > 0 : AFTER APPENDIX
    for _epoch in range(1,num_of_epochs+1):
        for _k in range(total_ks):
            paragraph_at_k = my_dict[str(_epoch)]['top_k'][str(total_ks)]['closest_labels_after'][question][_k]
            q_to_p_distance_at_k = my_dict[str(_epoch)]['top_k'][str(total_ks)]['distances_after'][question][_k]
            is_ground_truth_in_k_range = my_dict[str(_epoch)]['top_k'][str(_k+1)]['are_founds_after'][question][0]
            questions.append((question,
                             _epoch,
                             _k+1,
                             paragraph_at_k, 
                             q_to_p_distance_at_k,
                             is_ground_truth_in_k_range
            ))
df_traced_questions = pd.DataFrame(questions, columns=['question_id','epoch', 'k', 'paragraph_id', 'distance_to_paragraph', 'is_ground_truth_in_k_range'])   

**********
Question 4257


In [46]:
question_id_to_examine = 4257

print('Question {} is displayed with itsprogress within all epoch and k values'.format(question_id_to_examine))
question_to_display_progress = df_traced_questions[df_traced_questions['question_id'] == question_id_to_examine]
# question_to_display_progress

Question 4257 is displayed with itsprogress within all epoch and k values


In [47]:
print('Question {} is plotted'.format(question_id_to_examine))
paragraphs_to_plot = set(question_to_display_progress['paragraph_id'])
data = []
for p in paragraphs_to_plot:
    df = question_to_display_progress[question_to_display_progress['paragraph_id'] == p].sort_values(by=['epoch', 'k'], ascending=[True, True])
    if df.shape[0] > num_of_epochs:
        name = 'Ground Truth {}'.format(p)
    else:
        name = 'Paragraph {}'.format(p)
    df = df.drop_duplicates(subset=['question_id', 'epoch'])
    plot_data = []
    for _epoch in range(num_of_epochs + 1):
        try:
            distance = df[df['epoch'] == _epoch]['distance_to_paragraph'].values[0]
        except:
            distance = 1.5
        plot_data.append((_epoch, distance))
    plot_df = pd.DataFrame(plot_data, columns=['epoch', 'distance'])  
    trace = go.Scatter(
    x = plot_df.index,
    y = plot_df['distance'],
    mode = 'lines+markers',
    name = name)
    data.append(trace)

layout = go.Layout(
    title='Distances of the paragraphs to that question {} are getting progressed in each epoch'.format(question_id_to_examine),
    xaxis=dict(
        title='Epochs'
    ),
    yaxis=dict(
        title='Distance'
    )
)
fig =  go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

Question 4257 is plotted


In [285]:
"""
What is the progress of the paragraphs in which they became a ground-truth of another questions?
# First we need to find which questions were assigned to that paragraph
"""

paragraph_ids_to_examine = [15135]
questions = []
for p in paragraph_ids_to_examine:
    try:
        print('*' * 10)
        print('Parahraph {}'.format(p)) 
         # FOR EPOCH 0 : BEFORE APPENDIX
        _epoch = 0
        for _k in range(total_ks):
            try:
                question = np.where(data_dict['top_k'][str(_k+1)]['closest_labels_before'] 
                                                    == p)[0][_k]
                paragraph_at_k = my_dict['top_k'][str(total_ks)]['closest_labels_before'][question][_k]
                q_to_p_distance_at_k = my_dict['top_k'][str(total_ks)]['distances_before'][question][_k]
                is_ground_truth_in_k_range = my_dict['top_k'][str(_k+1)]['are_founds_before'][question][0]
                questions.append((question,
                                 _epoch,
                                 _k+1,
                                 paragraph_at_k, 
                                 q_to_p_distance_at_k,
                                 is_ground_truth_in_k_range
                ))
            except:
                print('There is no such a question that belongs to paragraph {} in that 5000 subset at epoch:{}, k:{}'.format(p, _epoch, _k+1))

        # FOR EPOCH > 0 : AFTER APPENDIX
        for _epoch in range(1,num_of_epochs+1):
            for _k in range(total_ks):
                try:
                    paragraph_at_k = my_dict[str(_epoch)]['top_k'][str(total_ks)]['closest_labels_after'][question][_k]
                    q_to_p_distance_at_k = my_dict[str(_epoch)]['top_k'][str(total_ks)]['distances_after'][question][_k]
                    is_ground_truth_in_k_range = my_dict[str(_epoch)]['top_k'][str(_k+1)]['are_founds_after'][question][0]
                    questions.append((question,
                                     _epoch,
                                     _k+1,
                                     paragraph_at_k, 
                                     q_to_p_distance_at_k,
                                     is_ground_truth_in_k_range
                    ))
                except:
                    print('There is no such a question that belongs to paragraph {} in that 5000 subset at epoch:{}, k:{}'.format(p, _epoch, _k+1))
                
    except:
        print('There is no such a question that belongs to paragraph {} in that 5000 subset at epoch:{}, k:{}'.format(p, _epoch, _k+1))

df_traced_questions = pd.DataFrame(questions, columns=['question_id','epoch', 'k', 'paragraph_id', 'distance_to_paragraph', 'is_ground_truth_in_k_range'])   

**********
Parahraph 15135
There is no such a question that belongs to paragraph 15135 in that 5000 subset at epoch:0, k:1
There is no such a question that belongs to paragraph 15135 in that 5000 subset at epoch:0, k:2
There is no such a question that belongs to paragraph 15135 in that 5000 subset at epoch:0, k:3
There is no such a question that belongs to paragraph 15135 in that 5000 subset at epoch:0, k:4
There is no such a question that belongs to paragraph 15135 in that 5000 subset at epoch:0, k:5
There is no such a question that belongs to paragraph 15135 in that 5000 subset at epoch:0, k:6
There is no such a question that belongs to paragraph 15135 in that 5000 subset at epoch:0, k:7
There is no such a question that belongs to paragraph 15135 in that 5000 subset at epoch:0, k:8
There is no such a question that belongs to paragraph 15135 in that 5000 subset at epoch:0, k:9
There is no such a question that belongs to paragraph 15135 in that 5000 subset at epoch:0, k:10


In [167]:
"""
In each training iteration, Are questions moving closer to ground_truth or not
if a delta value is moving to smaller value, it means question is moving to farther from its ground-truth, 
else is moving to closer to its ground-truth. The bigger, the better.
"""

data_delta = pd.DataFrame()
for i in range(1,num_of_epochs+1):
    data_delta[i] = my_dict[str(i)]['delta_before_after_model'][question_ids]
data_delta = data_delta.T
data_delta.columns = [question_ids[col_name] for col_name in data_delta.columns]
data = []
for q in question_ids:
    trace = go.Scatter(
    x = data_delta.index,
    y = data_delta[q],
    mode = 'lines+markers',
    name = 'Question {}'.format(q))
    data.append(trace)

layout = go.Layout(
    title='Delta Before Trained Q. Embeds. to Ground Truth vs After Trained Q. Embeds. to Ground Truth',
    xaxis=dict(
        title='Epochs'
    ),
    yaxis=dict(
        title='Delta Distance'
    )
)
fig =  go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

In [145]:
"""
Standart Deviation of the distances q to ground truths for each epoch. If it is getting smaller, which means that 
embeddings are getting look like each other
"""

data_dev = []
data_dev.append(np.std(my_dict['distance_from_before_model_q_to_p']))
for i in range(1,num_of_epochs+1):
    data_dev.append(np.std(my_dict[str(i)]['distance_from_after_model_q_to_p']))
df_dev = pd.DataFrame(data=data_dev, columns=['std_dev'])
trace = go.Scatter(
x = df_dev.index,
y = df_dev['std_dev'],
mode = 'lines+markers')

layout = go.Layout(
    title='Standart Deviation of the distances q to ground truths for each epoch',
    xaxis=dict(
        title='Epochs'
    ),
    yaxis=dict(
        title='Standart Deviation of the distances'
    )
)
fig =  go.Figure(data=[trace], layout=layout)
py.offline.iplot(fig)

In [146]:
"""
The idea here is that how many pairs we have that their distance are getting worse means that
they are getting farther. 
"""
number_of_bad_questions = []
set_bad_questions = set()
for i in range(1,num_of_epochs+1):
    bad_questions = np.where(my_dict[str(i)]['distance_from_after_model_q_to_p'] < 0)
    bad_questions = my_dict[str(i)]['distance_from_after_model_q_to_p'][bad_questions]
    number_of_bad_questions.append(bad_questions.shape[0])
    set_bad_questions.update(bad_questions)
    
df_farther = pd.DataFrame(data=number_of_bad_questions, columns=['number_of_bad_questions'])
trace = go.Scatter(
x = df_farther.index,
y = df_farther['number_of_bad_questions'],
mode = 'lines+markers')

layout = go.Layout(
    title='How many questions are getting farther from its pair paragraph',
    xaxis=dict(
        title='Epochs'
    ),
    yaxis=dict(
        title='Number of bad questions'
    )
)
fig =  go.Figure(data=[trace], layout=layout)
py.offline.iplot(fig)